In [6]:
def img_2_out(image,prompt):
    inputs = processor(prompt, image, return_tensors="pt").to("cuda")
    # autoregressively complete prompt
    output = model.generate(**inputs, max_new_tokens=25)
    return processor.decode(output[0], skip_special_tokens=True)

In [7]:
def StripAnswer(string):
    # Define the pattern to match everything after "assistant"
    pattern = r'assistant\s*(.*)'
    
    # Use regular expressions to find the text after "assistant"
    match = re.search(pattern, string, re.DOTALL)  # Use re.DOTALL to match across line breaks
    
    if match:
        response = match.group(1).strip()  # Use .strip() to remove leading/trailing whitespace
        print(response)
        return response
    else:
        print("No response found.")
        return None

In [8]:
import pickle
def save_dict(file_name,dict)->None:
    dbfile = open(file_name, 'wb')
    pickle.dump(dict, dbfile)                    
    dbfile.close()

In [9]:
import requests
import re
from PIL import Image
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_id = "PerRing/llava-v1.6-34b-hf"
model = LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map='auto',
    low_cpu_mem_usage=True, 
)

processor = AutoProcessor.from_pretrained(model_id)

Q='Are these characters touching. Only answer yes or no. Answer:'
prompt = f"""<|im_start|>system
Answer the questions.<|im_end|><|im_start|>user
<image>
{Q}<|im_end|><|im_start|>assistant
"""
"""
image_file = "https://images.pexels.com/photos/757889/pexels-photo-757889.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2"

raw_image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(prompt, raw_image, return_tensors='pt').to(0, torch.float16)

output = model.generate(**inputs, max_length=256, temperature=0.4, do_sample=True)
print(processor.decode(output[0], skip_special_tokens=True))
"""


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

'\nimage_file = "https://images.pexels.com/photos/757889/pexels-photo-757889.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2"\n\nraw_image = Image.open(requests.get(image_file, stream=True).raw)\ninputs = processor(prompt, raw_image, return_tensors=\'pt\').to(0, torch.float16)\n\noutput = model.generate(**inputs, max_length=256, temperature=0.4, do_sample=True)\nprint(processor.decode(output[0], skip_special_tokens=True))\n'

In [ ]:
img_file = "unity_images/non.png"
img_non = Image.open(img_file)

img_file = "unity_images/con.png"
img_con = Image.open(img_file)

In [ ]:
image_list =[]
image_list.append(img_non)
image_list.append(img_con)


In [ ]:
import os
from PIL import Image

def read_images_from_folder(folder_path):
    image_list = []
    
    for category in os.listdir(folder_path):
        category_path = os.path.join(folder_path, category)
        if os.path.isdir(category_path):
            category_label = category.split('_')[0]  # Extracting category label (Consensual or Nonconsensual)
            for filename in os.listdir(category_path):
                if filename.endswith(".png"):  # Assuming all images are PNG format
                    img_path = os.path.join(category_path, filename)
                    img_id = os.path.splitext(filename)[0]  # Extracting image ID from filename
                    image = Image.open(img_path)
                    image_dict = {"img_id": img_id, "img": image, "category": category_label}
                    image_list.append(image_dict)

    
    return image_list

# Example usage
folder_path = "/work/ree398/visual_research/unity_images/data"
image_list = read_images_from_folder(folder_path)
print(image_list[1])

In [10]:
Q='Are these characters touching. Only answer yes or no. Answer:'
prompt = f"""<|im_start|>system
Answer the questions.<|im_end|><|im_start|>user
<image>
{Q}<|im_end|><|im_start|>assistant
"""

img_2_out(img_con, prompt)

NameError: name 'img_con' is not defined

In [11]:
q = None

prompt = f"""<|im_start|>system
Answer the questions.<|im_end|><|im_start|>user
<image>
{q}<|im_end|><|im_start|>assistant
"""

question_expansion_list = ["Are characters touching? Only answer yes or no. Answer:",
                           "Are intimate areas being touching on a character? Only answer yes or no. Answer:"]

In [ ]:
description_prompt = """<|im_start|>system
    Provide short and concise response that focuses on character interactions.<|im_end|><|im_start|>user
    <image>
    Describe what is going on in this image?<|im_end|><|im_start|>assistant
    """

image_data_dict_list = []
for i, img in enumerate(image_list): 
    print(img['img_id'])
    image_data_dict = {"img_id":img['img_id'],
                       "img_category":img['category'],
                       "touching":None,
                       "initmate_area":None,
                       "scene_description":None}
    image = img['img']
    for q in question_expansion_list:
        prompt = f"""<|im_start|>system
        Answer the questions On respond with yes or no <|im_end|><|im_start|>user
        <image>
        {q}<|im_end|><|im_start|>assistant
        """
        if q == question_expansion_list[0]:
            image_data_dict['touching'] = StripAnswer(img_2_out(image, prompt))
        if q == question_expansion_list[1]:
            image_data_dict['initmate_area'] = StripAnswer(img_2_out(image, prompt))
    #image_data_dict['scene_description'] = StripAnswer(img_2_out(image, description_prompt))
    image_data_dict['scene_description'] = ""
    image_data_dict_list.append(image_data_dict)
    print(image_data_dict['img_category'])

    if i%50==0:
        print(i)
        save_dict('image_data_dict_list',image_data_dict_list)
save_dict('image_data_dict_list',image_data_dict_list)

In [12]:
# open a file, where you stored the pickled data
file = open('/work/ree398/visual_research/image_data_dict_list', 'rb')

# dump information to that file
data = pickle.load(file)
#print(data)
# close the file
file.close()

In [13]:
data

[{'img_id': 'Consensual_6',
  'img_category': 'C',
  'touching': 'No',
  'initmate_area': 'No',
  'scene_description': ''},
 {'img_id': 'Consensual_72',
  'img_category': 'C',
  'touching': 'No',
  'initmate_area': 'No',
  'scene_description': ''},
 {'img_id': 'Consensual_73',
  'img_category': 'C',
  'touching': 'Yes',
  'initmate_area': 'No',
  'scene_description': ''},
 {'img_id': 'Consensual_34',
  'img_category': 'C',
  'touching': 'No',
  'initmate_area': 'No',
  'scene_description': ''},
 {'img_id': 'Consensual_56',
  'img_category': 'C',
  'touching': 'No',
  'initmate_area': 'No',
  'scene_description': ''},
 {'img_id': 'Consensual_62',
  'img_category': 'C',
  'touching': 'Yes',
  'initmate_area': 'No',
  'scene_description': ''},
 {'img_id': 'Consensual_63',
  'img_category': 'C',
  'touching': 'Yes',
  'initmate_area': 'No',
  'scene_description': ''},
 {'img_id': 'Consensual_21',
  'img_category': 'C',
  'touching': 'Yes',
  'initmate_area': 'No',
  'scene_description': ''

In [24]:
fine_tune_data_list=[]
question_expansion_list = ["Are characters touching? Only answer yes or no. Answer:",
                           "Are intimate areas being touching on a character? Only answer yes or no. Answer:"]
for obj in data:
    sample_data = {"id":None,
                   "image":None,
                   "conversations":None       
    }
    
    id = obj['img_id']
    if obj['img_category']== 'C':
        image = f"/unity_images/data/C_0/{id}.png"
    elif obj['img_category']== 'NC':
         image = f"/unity_images/data/NC_0/{id}.png"   

    answers = [obj['touching'],
              obj['initmate_area']]
    for i, q in enumerate(question_expansion_list):
        conversation_h = {"from":"human",
                          "value":f"<image>\n{q}"}
        conversation_g = {"from":"gpt",
                          "value":f"{answers[i]}"}
        conversations = [conversation_h,
                         conversation_g]
        sample_data['id']=id
        sample_data['image']=image
        sample_data['conversations']=conversations
        conversations=[]
        fine_tune_data_list.append(sample_data)
                          














    

In [26]:
print(len(fine_tune_data_list))
fine_tune_data_list

286


[{'id': 'Consensual_6',
  'image': '/unity_images/data/C_0/Consensual_6.png',
  'conversations': [{'from': 'human',
    'value': '<image>\nAre intimate areas being touching on a character? Only answer yes or no. Answer:'},
   {'from': 'gpt', 'value': 'No'}]},
 {'id': 'Consensual_6',
  'image': '/unity_images/data/C_0/Consensual_6.png',
  'conversations': [{'from': 'human',
    'value': '<image>\nAre intimate areas being touching on a character? Only answer yes or no. Answer:'},
   {'from': 'gpt', 'value': 'No'}]},
 {'id': 'Consensual_72',
  'image': '/unity_images/data/C_0/Consensual_72.png',
  'conversations': [{'from': 'human',
    'value': '<image>\nAre intimate areas being touching on a character? Only answer yes or no. Answer:'},
   {'from': 'gpt', 'value': 'No'}]},
 {'id': 'Consensual_72',
  'image': '/unity_images/data/C_0/Consensual_72.png',
  'conversations': [{'from': 'human',
    'value': '<image>\nAre intimate areas being touching on a character? Only answer yes or no. Answ

In [29]:
import random
import json
# Shuffle the data
random.shuffle(fine_tune_data_list)

# Calculate the index to split the data
split_index = int(len(fine_tune_data_list) * 0.9)

# Split the data into train and test sets
train_data = fine_tune_data_list[:split_index]
test_data = fine_tune_data_list[split_index:]

# Verify the lengths of train and test sets
print("Train set length:", len(train_data))
print("Test set length:", len(test_data))

# Optionally, save train and test sets to separate files
with open('train_data.json', 'w') as train_file:
    json.dump(train_data, train_file)

with open('test_data.json', 'w') as test_file:
    json.dump(test_data, test_file)

Train set length: 257
Test set length: 29


In [ ]:
save_dict('fine_tune_data_list',fine_tune_data_list)

In [ ]:
json_object = json.dumps(fine_tune_data_list, indent=4)
 
# Writing to sample.json
with open("ft_data.json", "w") as outfile:
    outfile.write(json_object)